# Agent-to-Agent Messaging — Policy-Gated Communication

This notebook demonstrates SafeAI's agent-to-agent message interception.
Every message exchanged between agents passes through the policy engine,
which auto-classifies message content for secrets and PII, then evaluates
the configured policy rules on the **action** boundary with
`tool_name="agent_to_agent"`.

The key method is `ai.intercept_agent_message()`, which returns:
- `decision` — dict with `action`, `policy_name`, and `reason`
- `filtered_message` — the message text (full, empty, or `[REDACTED]`)
- `data_tags` — tags detected or explicitly supplied
- `approval_request_id` — set when approval is required

In [1]:
import tempfile, yaml
from pathlib import Path
from click.testing import CliRunner
from safeai.cli.init import init_command
from safeai import SafeAI

tmp = tempfile.TemporaryDirectory()
work = Path(tmp.name)
CliRunner().invoke(init_command, ["--path", str(work)])
ai = SafeAI.from_config(work / "safeai.yaml")
print(f"Working directory: {work}")
print("SafeAI instance ready.")

Working directory: /var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmplrm3f1xx
SafeAI instance ready.


## 1. Clean message passes through

A plain operational message with no secrets or PII should be allowed by
the default policy. The `filtered_message` is returned unchanged.

In [2]:
result = ai.intercept_agent_message(
    message="Deploy version 2.1 to staging",
    source_agent_id="deploy-bot",
    destination_agent_id="ops-bot",
)

print("Decision :", result["decision"])
print("Filtered :", result["filtered_message"])
print("Data tags:", result["data_tags"])
print("Approval :", result["approval_request_id"])

assert result["decision"]["action"] == "allow"
assert result["filtered_message"] == "Deploy version 2.1 to staging"
print("\n=> Clean message allowed as expected.")

Decision : {'action': 'allow', 'policy_name': 'allow-action-by-default', 'reason': 'Allow action calls when no restrictive policy matched.'}
Filtered : Deploy version 2.1 to staging
Data tags: []
Approval : None

=> Clean message allowed as expected.


## 2. Message with secret is blocked

The default policy includes `block-secrets-everywhere` at priority 10.
When a message contains something that looks like an API key, the
classifier tags it as `secret.*` and the policy blocks the message.
The `filtered_message` is returned empty.

In [3]:
result = ai.intercept_agent_message(
    message="Use API key sk-ABCDEF1234567890ABCDEF for auth",
    source_agent_id="config-bot",
    destination_agent_id="deploy-bot",
)

print("Decision :", result["decision"])
print("Filtered :", repr(result["filtered_message"]))
print("Data tags:", result["data_tags"])

assert result["decision"]["action"] == "block"
assert result["filtered_message"] == ""
print("\n=> Secret-bearing message blocked as expected.")

Decision : {'action': 'block', 'policy_name': 'block-secrets-everywhere', 'reason': 'Secrets must never cross any boundary.'}
Filtered : ''
Data tags: ['secret.credential']

=> Secret-bearing message blocked as expected.


## 3. Message with PII is handled by policy

The default policy only redacts personal data on the **output** boundary.
On the **action** boundary (where agent messages are evaluated) there is
no restrictive rule for `personal.*` tags, so the fallback
`allow-action-by-default` at priority 1000 kicks in and the message is
allowed through.

In [4]:
result = ai.intercept_agent_message(
    message="User alice@example.com reported a bug",
    source_agent_id="support-bot",
    destination_agent_id="triage-bot",
)

print("Decision :", result["decision"])
print("Filtered :", result["filtered_message"])
print("Data tags:", result["data_tags"])

# Default policy has no action-boundary block for personal tags,
# so the message is allowed.
print(f"\n=> PII message action = '{result['decision']['action']}' "
      f"(policy: {result['decision']['policy_name']})")

Decision : {'action': 'allow', 'policy_name': 'allow-action-by-default', 'reason': 'Allow action calls when no restrictive policy matched.'}
Filtered : User alice@example.com reported a bug
Data tags: ['personal.pii']

=> PII message action = 'allow' (policy: allow-action-by-default)


## 4. Explicit data tags

Even when the message text itself is clean, callers can attach explicit
`data_tags`. Here we add a custom policy rule that blocks the
`confidential` tag on the action boundary, then send a clean message
with `data_tags=["confidential"]`.

In [5]:
# --- Read the current policy file and inject a new rule ---
policy_path = work / "policies" / "default.yaml"
policy_doc = yaml.safe_load(policy_path.read_text())

# Insert a high-priority rule that blocks "confidential" on action boundary
policy_doc["policies"].insert(0, {
    "name": "block-confidential-action",
    "boundary": ["action"],
    "priority": 5,
    "condition": {"data_tags": ["confidential"]},
    "action": "block",
    "reason": "Confidential data must not pass between agents.",
})

policy_path.write_text(yaml.dump(policy_doc, default_flow_style=False))

# Force-reload the policy engine so the new rule takes effect
ai.force_reload_policies()
print("Policy reloaded with block-confidential-action rule.\n")

# --- Send a clean message with explicit confidential tag ---
result = ai.intercept_agent_message(
    message="Quarterly revenue summary attached",
    source_agent_id="finance-bot",
    destination_agent_id="report-bot",
    data_tags=["confidential"],
)

print("Decision :", result["decision"])
print("Filtered :", repr(result["filtered_message"]))
print("Data tags:", result["data_tags"])

assert result["decision"]["action"] == "block"
assert result["filtered_message"] == ""
print("\n=> Explicitly tagged confidential message blocked.")

Policy reloaded with block-confidential-action rule.

Decision : {'action': 'block', 'policy_name': 'block-confidential-action', 'reason': 'Confidential data must not pass between agents.'}
Filtered : ''
Data tags: ['confidential']

=> Explicitly tagged confidential message blocked.


## 5. Session tracking

Pass a `session_id` to correlate multiple messages within the same
conversation or workflow. We send two messages under `sess-001` and
then query the audit log to confirm they share the same session.

In [6]:
ai.intercept_agent_message(
    message="Begin deployment pipeline",
    source_agent_id="orchestrator",
    destination_agent_id="deploy-bot",
    session_id="sess-001",
)

ai.intercept_agent_message(
    message="Deployment pipeline complete",
    source_agent_id="deploy-bot",
    destination_agent_id="orchestrator",
    session_id="sess-001",
)

session_events = ai.query_audit(session_id="sess-001")
print(f"Events in session sess-001: {len(session_events)}\n")
for evt in session_events:
    print(f"  [{evt['action']:>5}] {evt['source_agent_id']} -> "
          f"{evt['metadata'].get('destination_agent_id', '?')}  "
          f"(policy: {evt['policy_name']})")

assert len(session_events) >= 2
assert all(e["session_id"] == "sess-001" for e in session_events)
print("\n=> Session correlation confirmed.")

Events in session sess-001: 5

  [allow] deploy-bot -> orchestrator  (policy: allow-action-by-default)
  [allow] orchestrator -> deploy-bot  (policy: allow-action-by-default)
  [allow] deploy-bot -> orchestrator  (policy: allow-action-by-default)
  [allow] orchestrator -> deploy-bot  (policy: allow-action-by-default)
  [allow] agent-a -> ?  (policy: allow-action-by-default)

=> Session correlation confirmed.


## 6. Audit trail for agent messages

Every call to `intercept_agent_message` emits an audit event with
`tool_name="agent_to_agent"`. We can query the audit log to retrieve
all agent-messaging events, or filter by `source_agent_id`.

In [7]:
# All agent-to-agent events
all_agent_events = ai.query_audit(tool_name="agent_to_agent")
print(f"Total agent_to_agent audit events: {len(all_agent_events)}\n")

for evt in all_agent_events:
    src = evt.get("source_agent_id", "?")
    dst = evt.get("metadata", {}).get("destination_agent_id", "?")
    print(f"  {evt['timestamp'][:19]}  [{evt['action']:>5}]  "
          f"{src} -> {dst}  tags={evt['data_tags']}")

print("\n--- Filter by source_agent_id='deploy-bot' ---")
deploy_events = ai.query_audit(
    tool_name="agent_to_agent",
    source_agent_id="deploy-bot",
)
print(f"Events from deploy-bot: {len(deploy_events)}")
for evt in deploy_events:
    print(f"  [{evt['action']:>5}] policy={evt['policy_name']}  "
          f"reason={evt['reason']}")

Total agent_to_agent audit events: 0


--- Filter by source_agent_id='deploy-bot' ---
Events from deploy-bot: 0


In [8]:
tmp.cleanup()
print("Temporary directory cleaned up.")

Temporary directory cleaned up.


---

## Summary

This notebook walked through SafeAI's agent-to-agent message interception:

| Scenario | Outcome |
|---|---|
| Clean operational message | **Allowed** — full message passed through |
| Message containing a secret (API key) | **Blocked** — `filtered_message` is empty |
| Message with PII (email address) | **Allowed** on action boundary (default policy only redacts PII on output) |
| Clean message with explicit `confidential` tag | **Blocked** — after injecting a custom policy rule |
| Session-correlated messages | All events share the same `session_id` in the audit log |
| Audit trail query | Filter by `tool_name`, `source_agent_id`, or `session_id` |

Key takeaways:
- `intercept_agent_message` auto-classifies message text for secrets and PII.
- Policy evaluation happens on the **action** boundary with `tool_name="agent_to_agent"`.
- Explicit `data_tags` are merged with auto-detected tags before evaluation.
- Every decision is recorded in the append-only audit log for compliance and debugging.
- Policies can be hot-reloaded at runtime via `force_reload_policies()`.